In [ ]:
import sys

sys.path.append("../..")
from fourier_scaffold import FourierScaffold
from vectorhash_functions import generate_1d_gaussian_kernel

import math
import torch
import matplotlib.pyplot as plt

shapes = [(3, 3), (5, 5)]


def gen_Q_degenerate(n, N_patts):
    return torch.complex(
        torch.cat([torch.eye(n), torch.zeros((n, N_patts - n))], dim=1),
        torch.zeros(n, N_patts),
    )



def verify_frame_properties(D, n, epsilon, omega, N_patts):
    print(
        f"Verifying with D={D}, n={n}, epsilon={epsilon}, omega={omega}, N_patts={N_patts}"
    )
    if n >= D:
        print(
            "Note: The inverse failure is expected when n < D. The current configuration (n >= D) may result in a successful inverse calculation."
        )
    print("-" * 50)

    scaffold = FourierScaffold(
        shapes=torch.tensor(shapes), D=D, _skip_K_calc=True, _skip_gs_calc=True
    )
    Phi_n = scaffold.gbook()
    # print((Phi_n.conj().T @ Phi_n).abs())

    Q_n = gen_Q_degenerate(n, N_patts)
    H_noiseless = Phi_n @ Q_n.T

    print("Attempting to calculate the inverse of H_n H_n*...")
    matrix_to_invert = H_noiseless @ H_noiseless.T.conj()
    try:
        torch.linalg.inv(matrix_to_invert)
        print("Inverse calculation SUCCEEDED. This is expected if n >= D.")
    except torch.linalg.LinAlgError:
        print(
            "Inverse calculation FAILED. This confirms the theoretical result that H_n H_n* is non-invertible when n < D."
        )
    print("-" * 50)

    eigvals_noiseless = torch.linalg.eigvalsh(matrix_to_invert).flip(0)[:n]
    print(eigvals_noiseless)
    lower_bound_noiseless = max(1 - (n - 1) * epsilon, 0)
    upper_bound_noiseless = 1 + (n - 1) * epsilon

    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.hist(eigvals_noiseless, bins=20, color="skyblue", edgecolor="black")
    plt.axvline(
        lower_bound_noiseless,
        color="red",
        linestyle="--",
        label="Theoretical Lower Bound",
    )
    plt.axvline(
        upper_bound_noiseless,
        color="red",
        linestyle="--",
        label="Theoretical Upper Bound",
    )
    plt.title("Eigenvalue Spectrum of Noiseless $H_nH_n^*$")
    plt.xlabel("Eigenvalue")
    plt.ylabel("Frequency")
    plt.legend()
    plt.grid(True)

    delta_H = torch.randn(D, n)

    norm_bound = math.sqrt(n * epsilon * omega)
    if torch.linalg.norm(delta_H, "fro") > 0:
        delta_H = delta_H / torch.linalg.norm(delta_H, "fro") * norm_bound

    H_noisy = H_noiseless + delta_H

    matrix_to_invert_noisy = H_noisy @ H_noisy.T.conj()
    eigvals_noisy = torch.linalg.eigvalsh(matrix_to_invert_noisy).flip(0)[:n]
    print(eigvals_noisy)
    lower_bound_noisy = max(1 - (n - 1) * epsilon - math.sqrt(n) * epsilon * omega, 0)
    upper_bound_noisy = 1 + (n - 1) * epsilon + math.sqrt(n) * epsilon * omega

    plt.subplot(1, 2, 2)
    plt.hist(eigvals_noisy, bins=20, color="lightgreen", edgecolor="black")
    plt.axvline(
        lower_bound_noisy, color="blue", linestyle="--", label="Theoretical Lower Bound"
    )
    plt.axvline(
        upper_bound_noisy, color="blue", linestyle="--", label="Theoretical Upper Bound"
    )
    plt.title("Eigenvalue Spectrum of Noisy $\\tilde{H}_n\\tilde{H}_n^*$")
    plt.xlabel("Eigenvalue")
    plt.ylabel("Frequency")
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()


# D > n is required for inverse failure.
# D = 2000  # Dimension of vectors g(k)
# n = 50  # Number of patterns, n < D
# delta = 1 - 0.01
# epsilon = math.sqrt(
#     (2 / D) * math.log(2 / delta)
# )  # Small quasi-orthogonality perturbation
# Npatts = torch.prod(torch.tensor(shapes))
# omega = epsilon * Npatts  # A constant for the noise bound

# verify_frame_properties(D, n, epsilon, omega, Npatts)

In [ ]:
def gen_Q_gaussian(n, N_patts, sigma):
    print(n, N_patts, sigma)
    Q_real = torch.empty((n, N_patts))
    Q_imag = torch.zeros((n, N_patts))
    for j in range(n):
        shift = j + (N_patts - 1) // 2
        Q_real[j] = generate_1d_gaussian_kernel((N_patts - 1) // 2, 0, sigma).roll(
            shift
        )
    Q = torch.complex(Q_real, Q_imag)

    return Q

#print(gen_Q_gaussian(11, 225, 0.2))

In [ ]:
def verify_frame_properties_gaussian(D, n, epsilon, omega, N_patts):
    print(
        f"Verifying with D={D}, n={n}, epsilon={epsilon}, omega={omega}, N_patts={N_patts}"
    )
    if n >= D:
        print(
            "Note: The inverse failure is expected when n < D. The current configuration (n >= D) may result in a successful inverse calculation."
        )
    print("-" * 50)

    scaffold = FourierScaffold(
        shapes=torch.tensor(shapes), D=D, _skip_K_calc=True, _skip_gs_calc=True
    )
    Phi_n = scaffold.gbook()
    # print((Phi_n.conj().T @ Phi_n).abs())

    Q_n = gen_Q_gaussian(n, N_patts, 1)
    H_noiseless = Phi_n @ Q_n.T

    print("Attempting to calculate the inverse of H_n H_n*...")
    matrix_to_invert = H_noiseless @ H_noiseless.T.conj()
    try:
        torch.linalg.inv(matrix_to_invert)
        print("Inverse calculation SUCCEEDED. This is expected if n >= D.")
    except torch.linalg.LinAlgError:
        print(
            "Inverse calculation FAILED. This confirms the theoretical result that H_n H_n* is non-invertible when n < D."
        )
    print("-" * 50)

    eigvals_noiseless = torch.linalg.eigvalsh(matrix_to_invert).flip(0)[:n]
    print(eigvals_noiseless)
    lower_bound_noiseless = max(1 - (n - 1) * epsilon, 0)
    upper_bound_noiseless = 1 + (n - 1) * epsilon

    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.hist(eigvals_noiseless, bins=20, color="skyblue", edgecolor="black")
    # plt.axvline(
    #     lower_bound_noiseless,
    #     color="red",
    #     linestyle="--",
    #     label="Theoretical Lower Bound",
    # )
    # plt.axvline(
    #     upper_bound_noiseless,
    #     color="red",
    #     linestyle="--",
    #     label="Theoretical Upper Bound",
    # )
    plt.title("Eigenvalue Spectrum of Noiseless $H_nH_n^*$")
    plt.xlabel("Eigenvalue")
    plt.ylabel("Frequency")
    plt.legend()
    plt.grid(True)

    delta_H = torch.randn(D, n)

    norm_bound = math.sqrt(n * epsilon * omega)
    if torch.linalg.norm(delta_H, "fro") > 0:
        delta_H = delta_H / torch.linalg.norm(delta_H, "fro") * norm_bound

    H_noisy = H_noiseless + delta_H

    matrix_to_invert_noisy = H_noisy @ H_noisy.T.conj()
    eigvals_noisy = torch.linalg.eigvalsh(matrix_to_invert_noisy).flip(0)[:n]
    print(eigvals_noisy)
    lower_bound_noisy = max(1 - (n - 1) * epsilon - math.sqrt(n) * epsilon * omega, 0)
    upper_bound_noisy = 1 + (n - 1) * epsilon + math.sqrt(n) * epsilon * omega

    plt.subplot(1, 2, 2)
    plt.hist(eigvals_noisy, bins=20, color="lightgreen", edgecolor="black")
    # plt.axvline(
    #     lower_bound_noisy, color="blue", linestyle="--", label="Theoretical Lower Bound"
    # )
    # plt.axvline(
    #     upper_bound_noisy, color="blue", linestyle="--", label="Theoretical Upper Bound"
    # )
    plt.title("Eigenvalue Spectrum of Noisy $\\tilde{H}_n\\tilde{H}_n^*$")
    plt.xlabel("Eigenvalue")
    plt.ylabel("Frequency")
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()

    print(matrix_to_invert @ matrix_to_invert.pinverse())
    print(matrix_to_invert_noisy @ matrix_to_invert_noisy.pinverse())

def verify_frame_properties_gaussian_regularized(D, n, epsilon, omega, N_patts, llambda):
    print(
        f"Verifying with D={D}, n={n}, epsilon={epsilon}, omega={omega}, N_patts={N_patts}"
    )
    if n >= D:
        print(
            "Note: The inverse failure is expected when n < D. The current configuration (n >= D) may result in a successful inverse calculation."
        )
    print("-" * 50)

    scaffold = FourierScaffold(
        shapes=torch.tensor(shapes), D=D, _skip_K_calc=True, _skip_gs_calc=True
    )
    Phi_n = scaffold.gbook()
    # print((Phi_n.conj().T @ Phi_n).abs())

    Q_n = gen_Q_gaussian(n, N_patts, 1)
    H_noiseless = Phi_n @ Q_n.T

    print("Attempting to calculate the inverse of H_n H_n*...")
    matrix_to_invert = H_noiseless @ H_noiseless.T.conj() + llambda * torch.complex(
        torch.eye(D), torch.zeros(D, D)
    )
    try:
        torch.linalg.inv(matrix_to_invert)
        print("Inverse calculation SUCCEEDED. This is expected if n >= D.")
    except torch.linalg.LinAlgError:
        print(
            "Inverse calculation FAILED. This confirms the theoretical result that H_n H_n* is non-invertible when n < D."
        )
    print("-" * 50)

    eigvals_noiseless = torch.linalg.eigvalsh(matrix_to_invert).flip(0)[:n]
    print(eigvals_noiseless)
    lower_bound_noiseless = max(1 - (n - 1) * epsilon, 0)
    upper_bound_noiseless = 1 + (n - 1) * epsilon

    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.hist(eigvals_noiseless, bins=20, color="skyblue", edgecolor="black")
    # plt.axvline(
    #     lower_bound_noiseless,
    #     color="red",
    #     linestyle="--",
    #     label="Theoretical Lower Bound",
    # )
    # plt.axvline(
    #     upper_bound_noiseless,
    #     color="red",
    #     linestyle="--",
    #     label="Theoretical Upper Bound",
    # )
    plt.title("Eigenvalue Spectrum of Noiseless $H_nH_n^*$")
    plt.xlabel("Eigenvalue")
    plt.ylabel("Frequency")
    plt.legend()
    plt.grid(True)

    delta_H = torch.randn(D, n)

    norm_bound = math.sqrt(n * epsilon * omega)
    if torch.linalg.norm(delta_H, "fro") > 0:
        delta_H = delta_H / torch.linalg.norm(delta_H, "fro") * norm_bound

    H_noisy = H_noiseless + delta_H

    matrix_to_invert_noisy = H_noisy @ H_noisy.T.conj() + llambda * torch.complex(
        torch.eye(D), torch.zeros(D, D)
    )

    eigvals_noisy = torch.linalg.eigvalsh(matrix_to_invert_noisy).flip(0)[:n]
    print(eigvals_noisy)
    lower_bound_noisy = max(1 - (n - 1) * epsilon - math.sqrt(n) * epsilon * omega, 0)
    upper_bound_noisy = 1 + (n - 1) * epsilon + math.sqrt(n) * epsilon * omega

    plt.subplot(1, 2, 2)
    plt.hist(eigvals_noisy, bins=20, color="lightgreen", edgecolor="black")
    # plt.axvline(
    #     lower_bound_noisy, color="blue", linestyle="--", label="Theoretical Lower Bound"
    # )
    # plt.axvline(
    #     upper_bound_noisy, color="blue", linestyle="--", label="Theoretical Upper Bound"
    # )
    plt.title("Eigenvalue Spectrum of Noisy $\\tilde{H}_n\\tilde{H}_n^*$")
    plt.xlabel("Eigenvalue")
    plt.ylabel("Frequency")
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()

    print(matrix_to_invert @ matrix_to_invert.pinverse())
    print(matrix_to_invert_noisy @ matrix_to_invert_noisy.pinverse())


# Run the verification with example parameters
# D > n is required for inverse failure.
D = 2000  # Dimension of vectors g(k)
n = 50  # Number of patterns, n < D
delta = 1 - 0.01
epsilon = math.sqrt(
    (2 / D) * math.log(2 / delta)
)  # Small quasi-orthogonality perturbation
Npatts = torch.prod(torch.tensor(shapes))
omega = epsilon * Npatts  # A constant for the noise bound

verify_frame_properties_gaussian(D, n, epsilon, omega, Npatts.item())
verify_frame_properties_gaussian_regularized(D, n, epsilon, omega, Npatts.item(), llambda=1)